In [1]:
import polars as pl

In [2]:
dfOutputs = pl.read_csv("../data/outputs.csv")

In [3]:
import re

def isNujna(text):
    if re.search('Nujna prometna informacija', text, re.IGNORECASE):
        return 1
    if re.search('Nujne prometne informacije', text, re.IGNORECASE):
        return 1
    
    return 0

def isNova(text):
    if re.search('Nova prometna informacija', text, re.IGNORECASE):
        return 1
    if re.search('Nove prometne informacije', text, re.IGNORECASE):
        return 1
    
    return 0

In [4]:
# extract things like 1., 2. in 3. program

def extractPrograms(text):
    pattern = r'(\d+\.)\s*(?:in|,|&)?\s*(?=\d+\.|program)'

    matches = re.findall(pattern, text)

    m = []

    for match in matches:
        p = match.strip().replace(".", "")
        if int(p) <= 3:
            m.append(p)

    return ",".join(m)

In [5]:
dfOutputs = dfOutputs.with_columns([
    (pl.col("content").map_elements(lambda x: isNujna(x), return_dtype=pl.Int8).alias('nujna')),
    (pl.col("content").map_elements(lambda x: isNova(x), return_dtype=pl.Int8).alias('nova')),
    (pl.col("content").map_elements(lambda x: extractPrograms(x), return_dtype=pl.String).alias('programs')),
])

In [8]:
dfOutputs = dfOutputs.select(["id", "date", "time", "nujna", "nova", "programs", "content", "file_name", "year", "month"])

In [9]:
dfOutputs.head()

id,date,time,nujna,nova,programs,content,file_name,year,month
str,str,str,i8,i8,str,str,str,i64,str
"""f16895dd-6f45-50da-8ce6-966d44…","""2022-01-01""","""06:00:00""",0,0,"""01,1,2""","""Prometne informacije 01.…","""TMP-808""",2022,"""Januar"""
"""6e542c9c-2559-5837-9c0b-23b455…","""2022-01-01""","""06:30:00""",0,0,"""01,1,2""","""Prometne informacije 01.…","""TMP-807""",2022,"""Januar"""
"""376bf2cd-fd6a-51ff-bed5-ad53a4…","""2022-01-01""","""07:00:00""",0,0,"""01,1,2""","""Prometne informacije 01.…","""TMP-806""",2022,"""Januar"""
"""ba3714f5-35dc-539d-8c25-7da7f9…","""2022-01-01""","""08:00:00""",0,0,"""01,1,3""","""Prometne informacije 01.…","""TMP-805""",2022,"""Januar"""
"""3db9bb19-c329-5d58-a723-039465…","""2022-01-01""","""08:30:00""",0,0,"""01,2""","""Prometne informacije 01.…","""TMP-804""",2022,"""Januar"""


In [10]:
dfOutputs.write_csv("../data/outputs.csv")